In [ ]:
# lightgbm 설치
# 버전 확인해 보고 코드 수정
! pip install lightgbm==4.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.6 MB/s eta 0:00:00


In [ ]:
! python --version

Python 3.11.11


In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
google                             2.0.3
numpy                              1.26.4
pandas                             2.2.2
sklearn 1.6.1
'''

In [ ]:
! pip list

Package                            Version
---------------------------------- ------------------
absl-py                            1.4.0
accelerate                         1.3.0
aiohappyeyeballs                   2.4.6
aiohttp                            3.11.13
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.4
ale-py                             0.10.2
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.6.0
arviz                              0.20.0
astropy                            7.0.1
astropy-iers-data                  0.2025.2.24.0.34.4
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              25.1.0
audioread            

In [ ]:
import pandas as pd
import numpy as np

# 데이터 불러올 경로
train_path = "/content/drive/MyDrive/Aimers2502/train.csv"
test_path = "/content/drive/MyDrive/Aimers2502/test.csv"

# 데이터 불러오기
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

#ID 컬럼 제거
df_train = df_train.drop(columns=["ID"])
df_test = df_test.drop(columns=["ID"])

#데이터 크기 확인
print(f"Train 데이터 크기: {df_train.shape}")
print(f"Test 데이터 크기: {df_test.shape}")

Train 데이터 크기: (256351, 68)
Test 데이터 크기: (90067, 67)


In [ ]:
# 시술 당시 컬럼 라벨 인코딩
# '만18-34세': 0, '만35-37세': 1, '만38-39세': 2, '만40-42세': 3, '만43-44세': 4, '만45-50세': 5
# lgbm을 쓰는 만큼 '알 수 없음'은 결측치 유지

df_train['시술 당시 나이'] = df_train['시술 당시 나이'].replace({
    '만18-34세': 0,
    '만35-37세': 1,
    '만38-39세': 2,
    '만40-42세': 3,
    '만43-44세': 4,
    '만45-50세': 5,
    '알 수 없음': np.nan
})

df_test['시술 당시 나이'] = df_test['시술 당시 나이'].replace({
    '만18-34세': 0,
    '만35-37세': 1,
    '만38-39세': 2,
    '만40-42세': 3,
    '만43-44세': 4,
    '만45-50세': 5,
    '알 수 없음': np.nan
})

<ipython-input-25-651c7a7179ae>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['시술 당시 나이'] = df_train['시술 당시 나이'].replace({
<ipython-input-25-651c7a7179ae>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['시술 당시 나이'] = df_test['시술 당시 나이'].replace({


In [ ]:
# One-Hot Encoding을 위한 카테고리 리스트
categories = ["현재 시술용", "배아 저장용", "기증용", "난자 저장용", "연구용"]

# 새로운 컬럼 생성 (각 카테고리가 존재하면 1, 없으면 0)
for cat in categories:
    df_train[cat] = df_train["배아 생성 주요 이유"].apply(lambda x: 1 if cat in str(x) else 0)

# 새로운 컬럼 생성 (각 카테고리가 존재하면 1, 없으면 0)
for cat in categories:
    df_test[cat] = df_test["배아 생성 주요 이유"].apply(lambda x: 1 if cat in str(x) else 0)

# 배아 생성 주요 이유 컬럼 제거
df_train = df_train.drop(columns=["배아 생성 주요 이유"])
df_test = df_test.drop(columns=["배아 생성 주요 이유"])

In [ ]:
# check 피처
# 특정 시술 유형 내에 ICSI, IVF, Unknown가 존재하면 1, 없으면 0

# Train 데이터 처리
df_train["check_ICSI"] = df_train["특정 시술 유형"].str.contains("ICSI", na=False).astype(int)
df_train["check_IVF"] = df_train["특정 시술 유형"].str.contains("IVF", na=False).astype(int)
df_train["check_Unknown"] = (df_train["특정 시술 유형"].str.strip() == "Unknown").astype(int)

# Test 데이터 처리
df_test["check_ICSI"] = df_test["특정 시술 유형"].str.contains("ICSI", na=False).astype(int)
df_test["check_IVF"] = df_test["특정 시술 유형"].str.contains("IVF", na=False).astype(int)
df_test["check_Unknown"] = (df_test["특정 시술 유형"].str.strip() == "Unknown").astype(int)

# ✅ ID 컬럼 제거
df_train = df_train.drop(columns=["특정 시술 유형"])
df_test = df_test.drop(columns=["특정 시술 유형"])

In [ ]:
import pandas as pd
import numpy as np

# 분모가 0이 되는 경우를 방지하기 위한 작은 값
epsilon = 1e-5

# (1) 배아 생성 효율: 총 생성 배아 수 / (수집된 신선 난자 수)
df_train['embryo_creation_efficiency'] = df_train['총 생성 배아 수'] / (df_train['수집된 신선 난자 수'] + epsilon)

# (2) 배아 이식 효율: 이식된 배아 수 / (총 생성 배아 수)
df_train['embryo_transfer_efficiency'] = df_train['이식된 배아 수'] / (df_train['총 생성 배아 수'] + epsilon)

# (3) 미세주입 효율: 미세주입에서 생성된 배아 수 / (미세주입된 난자 수)
df_train['microinjection_efficiency'] = df_train['미세주입에서 생성된 배아 수'] / (df_train['미세주입된 난자 수'] + epsilon)

# (4) 미세주입 후 저장 비율: 미세주입 후 저장된 배아 수 / (미세주입에서 생성된 배아 수)
df_train['post_microinjection_storage_ratio'] = df_train['미세주입 후 저장된 배아 수'] / (df_train['미세주입에서 생성된 배아 수'] + epsilon)

# (7) 저장 배아 효율: 저장된 배아 수 / (총 생성 배아 수)
df_train['embryo_freezing_efficiency'] = df_train['저장된 배아 수'] / (df_train['총 생성 배아 수'] + epsilon)

# (8) 해동 배아 비율: 해동된 배아 수 / (저장된 배아 수)
df_train['thawed_to_stored_ratio'] = df_train['해동된 배아 수'] / (df_train['저장된 배아 수'] + epsilon)

# (9) 파트너 정자 사용 비율: 파트너 정자와 혼합된 난자 수 / (혼합된 난자 수)
df_train['partner_sperm_ratio'] = df_train['파트너 정자와 혼합된 난자 수'] / (df_train['혼합된 난자 수'] + epsilon)

# (12) 배아 이식 경과일 구간화: '배아 이식 경과일'을 quantile 기반으로 구간 나누기
# 구간 경계를 quantile로 결정 (예: 33%, 66%)
min_val = df_train['배아 이식 경과일'].min()
q33 = df_train['배아 이식 경과일'].quantile(0.33)
q66 = df_train['배아 이식 경과일'].quantile(0.66)
max_val = df_train['배아 이식 경과일'].max()

bins = [min_val - 1, q33, q66, max_val + 1]
labels = ['short', 'medium', 'long']
df_train['embryo_transfer_days_bin'] = pd.cut(df_train['배아 이식 경과일'], bins=bins, labels=labels)

In [ ]:
import pandas as pd
import numpy as np

# 분모가 0이 되는 경우를 방지하기 위한 작은 값
epsilon = 1e-5

# (1) 배아 생성 효율: 총 생성 배아 수 / (수집된 신선 난자 수)
df_test['embryo_creation_efficiency'] = df_test['총 생성 배아 수'] / (df_test['수집된 신선 난자 수'] + epsilon)

# (2) 배아 이식 효율: 이식된 배아 수 / (총 생성 배아 수)
df_test['embryo_transfer_efficiency'] = df_test['이식된 배아 수'] / (df_test['총 생성 배아 수'] + epsilon)

# (3) 미세주입 효율: 미세주입에서 생성된 배아 수 / (미세주입된 난자 수)
df_test['microinjection_efficiency'] = df_test['미세주입에서 생성된 배아 수'] / (df_test['미세주입된 난자 수'] + epsilon)

# (4) 미세주입 후 저장 비율: 미세주입 후 저장된 배아 수 / (미세주입에서 생성된 배아 수)
df_test['post_microinjection_storage_ratio'] = df_test['미세주입 후 저장된 배아 수'] / (df_test['미세주입에서 생성된 배아 수'] + epsilon)

# (7) 저장 배아 효율: 저장된 배아 수 / (총 생성 배아 수)
df_test['embryo_freezing_efficiency'] = df_test['저장된 배아 수'] / (df_test['총 생성 배아 수'] + epsilon)

# (8) 해동 배아 비율: 해동된 배아 수 / (저장된 배아 수)
df_test['thawed_to_stored_ratio'] = df_test['해동된 배아 수'] / (df_test['저장된 배아 수'] + epsilon)

# (9) 파트너 정자 사용 비율: 파트너 정자와 혼합된 난자 수 / (혼합된 난자 수)
df_test['partner_sperm_ratio'] = df_test['파트너 정자와 혼합된 난자 수'] / (df_test['혼합된 난자 수'] + epsilon)

# (12) 배아 이식 경과일 구간화: '배아 이식 경과일'을 quantile 기반으로 구간 나누기
# 구간 경계를 quantile로 결정 (예: 33%, 66%)
min_val = df_test['배아 이식 경과일'].min()
q33 = df_test['배아 이식 경과일'].quantile(0.33)
q66 = df_test['배아 이식 경과일'].quantile(0.66)
max_val = df_test['배아 이식 경과일'].max()

bins = [min_val - 1, q33, q66, max_val + 1]
labels = ['short', 'medium', 'long']
df_test['embryo_transfer_days_bin'] = pd.cut(df_test['배아 이식 경과일'], bins=bins, labels=labels)

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# ✅ 범주형 컬럼 리스트 확인
cat_cols_train = df_train.select_dtypes(include=["object"]).columns
cat_cols_test = df_test.select_dtypes(include=["object"]).columns

# Train과 Test의 공통된 범주형 컬럼만 선택
common_cat_cols = list(set(cat_cols_train) & set(cat_cols_test))

# ✅ 안전한 Label Encoding 적용
for col in common_cat_cols:
    le = LabelEncoder()

    # Train 데이터 학습
    df_train[col] = le.fit_transform(df_train[col])

    # Test 데이터 변환 (Train에 없는 값은 -1로 처리)
    test_values = np.array(df_test[col])  # 원본 테스트 값 저장
    known_classes = set(le.classes_)  # 훈련 데이터에서 학습한 클래스 저장

    # 안전한 변환 수행
    test_encoded = np.array([
        le.transform([value])[0] if value in known_classes else -1 # (Train에 없는 값은 -1로 처리)
        for value in test_values
    ])

    # 변환된 값을 Test 데이터에 저장
    df_test[col] = test_encoded

In [ ]:
feature_selected = [
    'embryo_creation_efficiency',
    '시술 당시 나이',
    '시술 시기 코드',
    'embryo_transfer_efficiency',
    '배아 이식 경과일',
    '이식된 배아 수',
    'embryo_freezing_efficiency',
    'microinjection_efficiency',
    '수집된 신선 난자 수',
    'IVF 시술 횟수',
    '클리닉 내 총 시술 횟수',
    '혼합된 난자 수',
    '총 생성 배아 수',
    '미세주입된 난자 수',
    '파트너 정자와 혼합된 난자 수',
    'post_microinjection_storage_ratio',
    '총 시술 횟수',
    '총 임신 횟수',
    '미세주입에서 생성된 배아 수',
    '저장된 배아 수',
    '난자 기증자 나이',
    'partner_sperm_ratio',
    '임신 시도 또는 마지막 임신 경과 연수',
    '총 출산 횟수',
    'thawed_to_stored_ratio',
    '불임 원인 - 남성 요인',
    'embryo_transfer_days_bin',
    '미세주입 배아 이식 수',
    'DI 시술 횟수',
    '정자 기증자 나이',
    '불임 원인 - 난관 질환',
    'IVF 임신 횟수',
    '해동된 배아 수',
    '미세주입 후 저장된 배아 수',
    '불임 원인 - 배란 장애',
    'check_Unknown',
    '단일 배아 이식 여부',
    '기증용',
    '불명확 불임 원인',
    'IVF 출산 횟수',
    '기증자 정자와 혼합된 난자 수',
    '착상 전 유전 검사 사용 여부',
    '난자 혼합 경과일',
    '난자 출처',
    '정자 출처',
    '불임 원인 - 자궁내막증',
    'PGS 시술 여부',
    '배란 자극 여부',
    'check_ICSI',
    '배란 유도 유형',
    '해동 난자 수',
    '기증 배아 사용 여부',
    '착상 전 유전 진단 사용 여부',
    '난자 해동 경과일',
    '난자 채취 경과일',
    'DI 임신 횟수',
    'PGD 시술 여부',
    '부부 부 불임 원인',
    'check_IVF',
    'DI 출산 횟수',
    '부부 주 불임 원인',
    '여성 부 불임 원인',
    '여성 주 불임 원인',
    '남성 부 불임 원인',
    '배아 해동 경과일',
    '남성 주 불임 원인',
    '대리모 여부',
    '현재 시술용',
    '시술 유형',
    '신선 배아 사용 여부',
    '동결 배아 사용 여부',
    '배아 저장용'
]

In [ ]:
import lightgbm as lgb
# from sklearn.metrics import roc_auc_score
import numpy as np

# ✅ 데이터 준비
X = df_train[feature_selected]  # 입력 변수
y = df_train["임신 성공 여부"]  # 타겟 변수

# ✅ LightGBM 모델 설정
lgbm_model = lgb.LGBMClassifier()
lgbm_model = lgb.LGBMClassifier(
                                objective='binary',
                                metric = 'auc',
                                boosting_type = 'gbdt',
                                random_state=42,
                                scale_pos_weight = 1.2469509553827316,
                                learning_rate = 0.02149524140540219,
                                num_leaves = 16,
                                max_depth = 13,
                                min_child_samples = 27,
                                subsample = 0.6808936174607194,
                                colsample_bytree = 0.5140506756742775,
                                reg_alpha = 9.081327528794018,
                                reg_lambda = 8.9592839180652,
                                n_estimators = 1132,
                                verbosity = -1
                                )

# ✅ 모델 학습
lgbm_model.fit(X, y)

# ✅ 테스트 데이터에서 훈련 데이터와 동일한 컬럼만 유지
X_test = df_test[X.columns]  # df_train과 동일한 컬럼만 선택

# ✅ 테스트 데이터 예측 (확률값 반환)
df_test["prediction_score"] = lgbm_model.predict_proba(X_test)[:, 1]  # 클래스 1의 확률

In [ ]:
df_test["prediction_score"]

,prediction_score
0,0.001343
1,0.001307
2,0.182194
3,0.130158
4,0.538235
...,...
90062,0.001602
90063,0.345649
90064,0.538853
90065,0.237165


In [ ]:
# df_test["prediction_score"]

,prediction_score
0,0.001343
1,0.001307
2,0.182194
3,0.130158
4,0.538235
...,...
90062,0.001602
90063,0.345649
90064,0.538853
90065,0.237165


In [ ]:
# 제출용 샘플 불러오기
submission = pd.read_csv("/content/drive/MyDrive/Aimers2502/sample_submission.csv")

# 예측 결과를 컬럼("probability")에 업데이트
submission["probability"] = df_test["prediction_score"]

# 제출 파일 저장
submission.to_csv("submission_0224.csv", index=False)
print("✅ submission_0224.csv 파일 생성 완료!")

✅ submission_0224.csv 파일 생성 완료!
